In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [9]:
train_df = pd.read_csv('../Data/train_감정분류.csv', index_col=0)
test_df = pd.read_csv('../Data/test_감정분류.csv', index_col=0)

## EDA

In [16]:
train_df

,감정_대분류,감정_소분류,전체응답
0,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.
1,기쁨,신이 난,아내분이 출산을 하시는군요. 정말 축하드려요.
2,기쁨,신이 난,아 지금 정말 신이 나.
3,기쁨,신이 난,잘 된 일이네요.
4,기쁨,신이 난,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
...,...,...,...
228463,불안,스트레스 받는,많이 서운하셨겠어요. 친구들에게 서운함을 느꼈을 때 해결하는 방법이 있나요?
228464,불안,스트레스 받는,카페에 가서 대화하거나 같이 술 마시면서 이야기했던 것 같아.
228465,불안,스트레스 받는,대화로 문제를 해결해오셨던 거군요? 그렇다면 이번 일에는 어떻게 하실 예정인가요?
228466,불안,스트레스 받는,직접 서운한 감정을 친구에게 얘기하려고 해.


In [15]:
train_df.value_counts('감정_소분류')

감정_소분류
혼란스러운          7597
고립된            7038
걱정스러운          5865
우울한            4751
스트레스 받는        4663
분노             4637
짜증내는           4594
두려운            4376
실망한            4140
충격 받은          4136
편안한            4126
눈물이 나는         4126
노여워하는          4088
한심한            4025
슬픔             4008
툴툴대는           4004
후회되는           4002
기쁨             3976
불안             3970
괴로워하는          3954
만족스러운          3952
당혹스러운          3936
부끄러운           3927
신이 난           3862
감사하는           3858
안도             3854
구역질 나는         3850
억울한            3832
신뢰하는           3788
배신당한           3773
성가신            3746
악의적인           3712
죄책감의           3702
상처             3692
낙담한            3688
조심스러운          3670
비통한            3664
환멸을 느끼는        3626
남의 시선을 의식하는    3610
초조한            3569
가난한, 불우한       3558
좌절한            3542
회의적인           3536
취약한            3510
자신하는           3504
당황             3484
마비된            3475
외로운            3470
안달하는           3458
희생된          